<a href="https://colab.research.google.com/github/jameszlj/NLP_with_python/blob/master/fast_ai_text_multi_label.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
from fastai.text import *

In [4]:
!wget https://github.com/jameszlj/NLP_with_python/raw/master/input/test.csv
!wget https://github.com/jameszlj/NLP_with_python/raw/master/input/train.csv

--2019-10-12 14:56:09--  https://github.com/jameszlj/NLP_with_python/raw/master/input/test.csv
Resolving github.com (github.com)... 192.30.253.112
Connecting to github.com (github.com)|192.30.253.112|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://raw.githubusercontent.com/jameszlj/NLP_with_python/master/input/test.csv [following]
--2019-10-12 14:56:10--  https://raw.githubusercontent.com/jameszlj/NLP_with_python/master/input/test.csv
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 151.101.0.133, 151.101.64.133, 151.101.128.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|151.101.0.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 60354593 (58M) [text/plain]
Saving to: ‘test.csv.1’

test.csv.1          100%[===================>]  57.56M   164MB/s    in 0.4s    

2019-10-12 14:56:11 (164 MB/s) - ‘test.csv.1’ saved [60354593/60354593]

--2019-10-12 14:56:15--  https://github.

In [5]:
!pip install pytorch-transformers

In [0]:
from pytorch_transformers import BertTokenizer, BertForSequenceClassification
from sklearn.metrics import classification_report, confusion_matrix
from sklearn.model_selection import train_test_split
class BertFastaiTokenizer(BaseTokenizer):
    def __init__(self, tokenizer, max_seq_len=128, **kwargs):
        self.pretrained_tokenizer = tokenizer
        self.max_seq_len = max_seq_len

    def __call__(self, *args, **kwargs):
        return self

    def tokenizer(self, t):
        return ["[CLS]"] + self.pretrained_tokenizer.tokenize(t)[:self.max_seq_len - 2] + ["[SEP]"]


class MyNoTupleModel(BertForSequenceClassification):
    def forward(self, *args, **kwargs):
        return super().forward(*args, **kwargs)[0]

In [0]:
max_seq_len = 256
batch_size = 32
path = Path(".")
bert_model = "bert-base-uncased"

In [0]:
train, test = [pd.read_csv(path / fname) for fname in ["train.csv", "test.csv"]]
train, valid = train_test_split(train, random_state=2)

In [9]:
train.head(10)

,id,comment_text,toxic,severe_toxic,obscene,threat,insult,identity_hate
5400,0e65327b5bbee0a5,"""\n\nI think it is both a Khmer character and ...",0,0,0,0,0,0
122278,8e162cbf88bfc540,"""\nI have added information to the article cre...",0,0,0,0,0,0
115832,6b646f8e87da9b5f,"Thanks kmccoy, I could really use your help on...",0,0,0,0,0,0
125459,9f1ab871e0d50caf,"""\nI never heard that, and I can't find any re...",0,0,0,0,0,0
131403,befbff9b502b865f,"""\n\nWell objectively, 200 PKK militants train...",0,0,0,0,0,0
158699,f218b5761c26faaa,I was thinking just adding a bullet point to t...,0,0,0,0,0,0
75648,ca678d42144fbb2d,Your objections to that sentence are quite cor...,0,0,0,0,0,0
118733,7a8f77565e0df03e,Thanks for being late to the party \n\nbut I h...,1,1,1,0,1,0
115153,67b7eec36480cd1c,I give up trying to write a page today. You lo...,0,0,0,0,0,0
109009,46dd6c0f2cdca00c,"""\n\n Robots everywhere! \n\nI see you have me...",0,0,0,0,0,0


In [10]:
bert_tokenizer = BertTokenizer.from_pretrained(bert_model)
print(list(bert_tokenizer.vocab.items())[1000:1005])
bert_vocab = Vocab(list(bert_tokenizer.vocab.keys()))
tok_func = BertFastaiTokenizer(bert_tokenizer, max_seq_len=max_seq_len)

100%|██████████| 231508/231508 [00:00<00:00, 417852.72B/s]


[('"', 1000), ('#', 1001), ('$', 1002), ('%', 1003), ('&', 1004)]


In [11]:
bert_fastai_tokenizer = Tokenizer(tok_func=tok_func, pre_rules=[], post_rules=[])
label_cols = ["toxic", "severe_toxic", "obscene", "threat", "insult", "identity_hate"]
## preprocess databunch
databunch = TextClasDataBunch.from_df(path, train, valid, test,
                                      tokenizer=bert_fastai_tokenizer,
                                      vocab=bert_vocab,
                                      include_bos=False,
                                      include_eos=False,
                                      text_cols="comment_text",
                                      label_cols=label_cols,
                                      bs=batch_size,
                                      collate_fn=partial(pad_collate, pad_first=False, pad_idx=0),
                                      )
databunch.show_batch()

text,target
"[CLS] "" it does hurt the article . that ' s why i removed it . it implies that he was known by many names , which is wrong . it also clearly contra ##ven ##es the style guide . did you read that ? if you do , you ' ll see the following three examples : ( from fide ##l castro ) : fide ##l alejandro castro ru",
[CLS] while it may be true that there are thousands of trolls for richard ka ##stle and lang lang . . . there is a huge difference . although lang lang does ( visually ) look like an idiot when he performs with his weird facial expressions and over exaggerated face . . . he still happens to be a brilliant pianist ( in the pieces where he doesn ',
"[CLS] agent x ##2 : basically thanks - with a ' little ' more ( little has to be a bit more in this case . it just lies in the nature of the matter . ) first : english is not my native language , so i won ' t correct the article myself . i rely on others to do that for me , based on my suggestions",
"[CLS] good news to za ##hid ! catholics join to global jihad ! . you can wait endorsement from catholic wikipedia ##ns , in order to ex ##ter ##minate and lynch evangelical ##s ! . congratulations my dear mu ##jah ##eed ! . search home · who are we · local ministries · contact us main menu · home · feedback · recommend us · search · stories archive ·",
"[CLS] "" i reverted it back because i preferred the original version before rod ball ' s edit ##s . i asked if anyone else agreed with rod ball on this issue - having gotten no response , i assumed that other people either didn ' t care , or like me preferred the original . in addition , i believe that p ##ja ##co ##bi already reverted these particular",


In [12]:
bert_pretrained_model = MyNoTupleModel.from_pretrained(bert_model, num_labels=6)
loss_func = nn.BCEWithLogitsLoss()
learn = Learner(databunch,
                bert_pretrained_model,
                loss_func=loss_func,
                metrics=accuracy)

100%|██████████| 440473133/440473133 [00:39<00:00, 11174717.26B/s]


In [0]:
learn.lr_find()

epoch,train_loss,valid_loss,accuracy,time


In [0]:
learn.recorder.plot()

In [0]:
learn.fit_one_cycle(4, 3e-5)